
# LCEL(LangChain Expression Language)
- LCEL : 프롬프트 → LLM —> chain —> 프롬프트로 연결됨
- LCEL의 목표는 LangChain 사용자가 복잡한 데이터 흐름과 AI 작업을 보다 직관적으로 구현할 수 있도록 하는 것

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

# 프롬프트 정의

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template('{city}에서 가장 유명한 랜드마크는 무엇인가요?')

In [5]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({'city':'파리'})

/var/folders/19/rx9v5nfx6mvgj7s4njrn8yzm0000gn/T/ipykernel_38543/2676035287.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'city': '파리',
 'text': '파리에서 가장 유명한 랜드마크는 에펠탑(Eiffel Tower)입니다. 에펠탑은 프랑스의 수도인 파리를 대표하는 상징적인 건물로, 세계적으로 유명한 관광지로 알려져 있습니다. 현재는 파리의 랜드마크로서만 아니라 세계 7대 현대 미래 건축물 중 하나로도 손꼽히고 있습니다. 에펠탑은 1889년 파리 만백주년 기념으로 건설된 것으로, 그 특이한 디자인과 아름다운 야경은 매년 수백만의 관광객을 끌어모으고 있습니다.'}

- LCEL(LCEL(LangChain Expression Language)) 형태로 전환함

In [7]:
chain = prompt | llm
chain.invoke({'city':'파리'})

AIMessage(content='파리에서 가장 유명한 랜드마크는 에펠 탑입니다. 에펠 탑은 프랑스의 수도인 파리의 상징이자 세계적인 관광 명소로 유명하며, 매년 많은 관광객들이 방문합니다. 1889년에 건설된 이 탑은 현재도 파리의 스카이라인을 장식하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 33, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgRqrkqYJOKbT2z9unFFaEeTa3dcY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--83344908-959c-4a6b-a1ef-3b181b3aa05c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 126, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# SequentialChain 객체 활용
- 첫번째 프롬프트의 결과 값이 두 번째 프롬프트의 입력으로 들어가야 하는 것이 기본인데, 두 번 호출하지 않고 하나의 chain으로 만들 수 있습니다.

In [8]:
from langchain.chains import SequentialChain

# citydhk transport 는 결과
# landmark는 첫번째 prompt의 결과값으로 사용해야 하므로, 
# 가져올 때 오직 지역의 랜드마크 이름만 가져오도록 만들어야 함
prompt_1 = PromptTemplate.from_template('{city}에서 가장 유명한 랜드마크는 무엇인가요? 설명은 필요없고 딱 이름 하나만 알려주세요')
prompt_2 = PromptTemplate.from_template('{landmark}에 {transport}로 가려면 어떻게 가나요?')

# prompt_1은 city를 입력으로 받고, landmark를 출력, 그 output_key가 protmpt_2의 landmark에 대입됨
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='landmark', verbose=True) #verbose=True로 하면 결과값을 출력해줌
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [10]:
chain = SequentialChain(chains=[chain_1, chain_2], 
                         input_variables=['city', 'transport'])

# 딕셔너리로 넣기
response = chain.invoke({'city':'서울', 'transport':'지하철'})



> Entering new LLMChain chain...
Prompt after formatting:
서울에서 가장 유명한 랜드마크는 무엇인가요? 설명은 필요없고 딱 이름 하나만 알려주세요

> Finished chain.


In [11]:
response

{'city': '서울',
 'transport': '지하철',
 'text': '롯데월드로 지하철로 가는 방법은 다음과 같습니다.\n\n1. 주변에 가장 가까운 지하철역을 찾습니다. (예: 잠실역, 잠실새내역)\n2. 해당 역에서 2호선에 승차합니다.\n3. 잠실역인 경우 2호선을 타고 잠실역에서 하차하고 2번 출구로 나와 롯데월드로 바로 이동이 가능합니다.\n4. 잠실새내역인 경우 2호선을 타고 잠실새내역에서 하차하고 9번 출구로 나와 롯데월드로 이동합니다.\n\n지하철로 이동 시 해당 역에서 어떤 출구로 나가야 하는지, 환승해야 하는 노선이 있는지 등을 미리 확인하고 이동하시면 더 편리하게 롯데월드에 도착할 수 있습니다.'}